# XLE - Energy Select Sector SPDR Fund.
It’s an ETF that tracks large U.S. energy companies in the S&P 500 (e.g., integrated majors, refiners, E&Ps).
- Think of it as a benchmark for the U.S. energy sector.

#### How XLE impacts energy company stock prices
**Direct impact:** If a company is inside XLE, its stock moves with ETF flows.


#### When XLE rises:
- Energy sector sentiment is strong
- Capital flows into energy
- Component stocks often rise

#### When XLE falls:
- Sector outflows
- Component stocks usually decline

In [4]:
# ===================== CONFIG: XLE (Energy Sector Index) =====================
from pathlib import Path
import pandas as pd
import yfinance as yf

START_DATE = "2010-01-01"
XLE_TICKER = "XLE"

BASE_DIR   = Path(r"D:\MS_Data_Science_Thesis\Data_Extraction")
RAW_DIR    = BASE_DIR / "Downloaded_datasets"
CLEAN_DIR  = BASE_DIR / "Raw_Data_Folder"

RAW_XLE_CSV   = RAW_DIR   / "xle_yahoo_raw.csv"
CLEAN_XLE_CSV = CLEAN_DIR / "XLE_daily.csv"
# ===========================================================================


# ---------- ensure folders exist (safe if already created) ----------
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)


def fetch_xle_from_yahoo(ticker: str, start_date: str) -> pd.DataFrame:
    """
    Fetch daily prices for XLE from Yahoo Finance using yfinance.
    Returns a DataFrame with DatetimeIndex and OHLCV columns.
    """
    print(f"[xle] Fetching {ticker!r} from Yahoo Finance from {start_date} onward...")
    # Explicitly set auto_adjust=False so we keep 'Adj Close'
    df = yf.download(ticker, start=start_date, auto_adjust=False, progress=False)
    df = df.sort_index()
    return df


def _flatten_columns(cols):
    """
    Turn possible MultiIndex columns into simple string names.
    Example: ('Adj Close', 'XLE') -> 'Adj Close_XLE'
    """
    flat = []
    for c in cols:
        if isinstance(c, tuple):
            parts = [str(x) for x in c if x is not None and str(x) != ""]
            flat.append("_".join(parts))
        else:
            flat.append(str(c))
    return flat


def clean_xle_df(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Clean raw XLE OHLCV data:
      - flatten MultiIndex columns if needed
      - keep Adjusted Close as the sector index
      - reset index, rename columns
      - drop missing values
    Output: date, xle_adj_close
    """
    if df_raw.empty:
        print("[xle] WARNING: raw XLE dataframe is empty.")
        return df_raw.copy()

    df = df_raw.copy()

    # Flatten column names if they are tuples / MultiIndex
    df.columns = _flatten_columns(df.columns)

    # Reset index so date is a column
    df = df.reset_index()

    # Flatten again (index column may also become tuple in some cases)
    df.columns = _flatten_columns(df.columns)

    # First column should be the date
    date_col = df.columns[0]

    # Try to find an "Adj Close" column (possibly like 'Adj Close', 'Adj Close_XLE', etc.)
    adj_cols = [c for c in df.columns if "adj" in c.lower() and "close" in c.lower()]

    # Fallback to "Close" if adj not found
    if not adj_cols:
        adj_cols = [c for c in df.columns if c.lower() == "close"]

    if not adj_cols:
        raise ValueError(f"[xle] Could not find an 'Adj Close' or 'Close' column in: {df.columns.tolist()}")

    value_col = adj_cols[0]

    df = df.rename(
        columns={
            date_col: "date",
            value_col: "xle_adj_close",
        }
    )

    # Enforce dtypes
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])

    df["xle_adj_close"] = pd.to_numeric(df["xle_adj_close"], errors="coerce")
    df = df.dropna(subset=["xle_adj_close"])

    # Final tidy ordering
    df = df.sort_values("date")[["date", "xle_adj_close"]]

    return df


# ---------- run pipeline ----------
try:
    # 1. Fetch from Yahoo (raw)
    xle_raw = fetch_xle_from_yahoo(XLE_TICKER, START_DATE)

    # 2. Save raw as reference
    xle_raw.to_csv(RAW_XLE_CSV)
    print(f"[xle] Saved raw XLE prices to {RAW_XLE_CSV}")

    # 3. Clean and standardize
    xle_clean = clean_xle_df(xle_raw)

    # 4. Save clean daily CSV (long format: date + single value column)
    xle_clean.to_csv(CLEAN_XLE_CSV, index=False)
    print(f"[xle] Saved clean XLE index to {CLEAN_XLE_CSV}")

    # 5. Quick QA printout
    if not xle_clean.empty:
        dmin = xle_clean["date"].min().date()
        dmax = xle_clean["date"].max().date()
        print(f"[xle] rows={len(xle_clean)}  range={dmin} → {dmax}")
    else:
        print("[xle] WARNING: clean XLE dataset is empty after filtering/cleaning.")

except Exception as e:
    print("[xle] ERROR while fetching or processing XLE data:", repr(e))


[xle] Fetching 'XLE' from Yahoo Finance from 2010-01-01 onward...
[xle] Saved raw XLE prices to D:\MS_Data_Science_Thesis\Data_Extraction\Downloaded_datasets\xle_yahoo_raw.csv
[xle] Saved clean XLE index to D:\MS_Data_Science_Thesis\Data_Extraction\Raw_Data_Folder\XLE_daily.csv
[xle] rows=4052  range=2010-01-04 → 2026-02-11
